# INSTRUCTION: Just run

Imports

In [2]:
from pymilvus import connections, DataType, CollectionSchema, FieldSchema, Collection, Partition, utility

Connection

In [3]:
# Check if the connection already exists
if connections.has_connection('default'):
    connections.remove_connection('default')  # Disconnect if it exists

# Now, reconnect with your new configuration
connections.connect(alias='default', host='localhost', port='19530')

Dropping

In [4]:
for name in collection_names:
    try:
        utility.drop_collection(f"{name}")
    except Exception as e:
        print(f"Unable to drop collection {name}: {str(e)}")

utility.list_collections()

[]

Collection creation

In [5]:
collection_names = {
    'text_collection': ['uuid', 'text_id', 'text', 'embeds', 'media', 'link', 'partition_name'],
    'author_collection': ['uuid', 'author', 'embeds', 'partition_name'],
    'title_collection': ['uuid', 'title', 'embeds', 'partition_name'],
    'date_collection': ['uuid', 'date', 'embeds', 'partition_name'],
    'contact_collection': ['uuid', 'contact', 'embeds', 'partition_name'],
    'name_collection': ['uuid', 'name', 'embeds', 'partition_name'],
    'position_collection': ['uuid', 'position', 'embeds', 'partition_name'],
    'department_collection': ['uuid', 'department', 'embeds', 'partition_name'],
}

In [6]:
collections = {}

for name, fields in collection_names.items():
    dimension = 1024
    field_schemas = [
        FieldSchema(name="uuid", dtype=DataType.VARCHAR, is_primary=True, max_length=36),
            FieldSchema("partition_name", dtype=DataType.VARCHAR, max_length=5000),
            FieldSchema(name="embeds", dtype=DataType.FLOAT_VECTOR, dim=dimension)
    ]
    for field in fields:
        if field not in ["uuid", "embeds", "partition_name"]:  # these fields have already been added
            field_schemas.append(FieldSchema(name=field, dtype=DataType.VARCHAR, max_length=5000))

    schema = CollectionSchema(fields=field_schemas, description=f"collection description of the collection: {name}")

    collections[name] = Collection(name=name, schema=schema)


In [7]:
utility.list_collections()

['author_collection',
 'title_collection',
 'date_collection',
 'contact_collection',
 'name_collection',
 'position_collection',
 'department_collection',
 'text_collection']

Partition creation

In [8]:
partitions = {
    "documents_partition": ["text_collection", "author_collection", "title_collection", "date_collection"],
    "social_posts_partition": ["text_collection", "date_collection"],
    "contacts_partition": ["name_collection", "text_collection", "contact_collection", "department_collection"],
    "people_partition": ["text_collection","name_collection","position_collection","department_collection"]
}

In [9]:
for partition_name, collection_names in partitions.items():
    for collection_name in collection_names:
        collection = collections[collection_name]
        collection.create_partition(partition_name)


Collection list

In [10]:
utility.list_collections()

['contact_collection',
 'name_collection',
 'position_collection',
 'department_collection',
 'text_collection',
 'author_collection',
 'title_collection',
 'date_collection']

Partition list

In [11]:
for collection in collections.values():
    display(collection.partitions)

[{"name": "_default", "collection_name": "text_collection", "description": ""},
 {"name": "documents_partition", "collection_name": "text_collection", "description": ""},
 {"name": "social_posts_partition", "collection_name": "text_collection", "description": ""},
 {"name": "contacts_partition", "collection_name": "text_collection", "description": ""},
 {"name": "people_partition", "collection_name": "text_collection", "description": ""}]

[{"name": "_default", "collection_name": "author_collection", "description": ""},
 {"name": "documents_partition", "collection_name": "author_collection", "description": ""}]

[{"name": "_default", "collection_name": "title_collection", "description": ""},
 {"name": "documents_partition", "collection_name": "title_collection", "description": ""}]

[{"name": "_default", "collection_name": "date_collection", "description": ""},
 {"name": "documents_partition", "collection_name": "date_collection", "description": ""},
 {"name": "social_posts_partition", "collection_name": "date_collection", "description": ""}]

[{"name": "_default", "collection_name": "contact_collection", "description": ""},
 {"name": "contacts_partition", "collection_name": "contact_collection", "description": ""}]

[{"name": "_default", "collection_name": "name_collection", "description": ""},
 {"name": "contacts_partition", "collection_name": "name_collection", "description": ""},
 {"name": "people_partition", "collection_name": "name_collection", "description": ""}]

[{"name": "_default", "collection_name": "position_collection", "description": ""},
 {"name": "people_partition", "collection_name": "position_collection", "description": ""}]

[{"name": "_default", "collection_name": "department_collection", "description": ""},
 {"name": "contacts_partition", "collection_name": "department_collection", "description": ""},
 {"name": "people_partition", "collection_name": "department_collection", "description": ""}]

In [12]:
index_params = {
  "metric_type": "L2", # Euclidean distance
  "index_type": "FLAT", # FLAT index type
  "params": {} # No additional parameters needed for FLAT
}
for collection in collections.values():
    collection.create_index("embeds", index_params)